In [1]:
import pandas as pd
import numpy as np
import bs4 as bs
import urllib.request

In [2]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2018"

In [3]:
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [4]:
tables = soup.find_all('table',class_='wikitable sortable')

In [5]:
len(tables)

4

In [6]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))

In [7]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

In [8]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.,Ref.
0,JANUARY,5,Insidious: The Last Key,Universal Pictures / Blumhouse Productions / S...,Adam Robitel (director); Leigh Whannell (scree...,[2],NaN
1,JANUARY,5,The Strange Ones,Vertical Entertainment,Lauren Wolkstein (director); Christopher Radcl...,[3],NaN
2,JANUARY,5,Stratton,Momentum Pictures,"Simon West (director); Duncan Falconer, Warren...",[4],NaN
3,JANUARY,10,Sweet Country,Samuel Goldwyn Films,"Warwick Thornton (director); David Tranter, St...",[5],NaN
4,JANUARY,12,The Commuter,Lionsgate / StudioCanal / The Picture Company,Jaume Collet-Serra (director); Byron Willinger...,[6],NaN
...,...,...,...,...,...,...,...
269,DECEMBER,25,Holmes & Watson,Columbia Pictures / Gary Sanchez Productions,Etan Cohen (director/screenplay); Will Ferrell...,NaN,[164]
270,DECEMBER,25,Vice,Annapurna Pictures / Plan B Entertainment,Adam McKay (director/screenplay); Christian Ba...,NaN,[137]
271,DECEMBER,25,On the Basis of Sex,Focus Features,Mimi Leder (director); Daniel Stiepleman (scre...,NaN,[229]
272,DECEMBER,25,Destroyer,Annapurna Pictures,"Karyn Kusama (director); Phil Hay, Matt Manfre...",NaN,[262]


In [9]:
df_2018 = df[['Title','Cast and crew']]

In [10]:
!pip install tmdbv3api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
from tmdbv3api import TMDb
import json
tmdb = TMDb()
tmdb.api_key = '2d71ac9f8f76e2fe996cdc92cab9f9cd'

In [12]:
from tmdbv3api import Movie
import json
tmdb_movie = Movie() 

In [13]:
def ids(x):
  result = tmdb_movie.search(x)
  if not result:
    return np.NaN
  else:
    return result[0].id

In [15]:
df_2018['id'] = df_2018['Title'].map(lambda x: ids(str(x)))

<ipython-input-15-b080d919ec1a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2018['id'] = df_2018['Title'].map(lambda x: ids(str(x)))


In [16]:
df_2018

,Title,Cast and crew,id
0,Insidious: The Last Key,Adam Robitel (director); Leigh Whannell (scree...,406563
1,The Strange Ones,Lauren Wolkstein (director); Christopher Radcl...,426258
2,Stratton,"Simon West (director); Duncan Falconer, Warren...",348389
3,Sweet Country,"Warwick Thornton (director); David Tranter, St...",468210
4,The Commuter,Jaume Collet-Serra (director); Byron Willinger...,399035
...,...,...,...
269,Holmes & Watson,Etan Cohen (director/screenplay); Will Ferrell...,426563
270,Vice,Adam McKay (director/screenplay); Christian Ba...,82
271,On the Basis of Sex,Mimi Leder (director); Daniel Stiepleman (scre...,339380
272,Destroyer,"Karyn Kusama (director); Phil Hay, Matt Manfre...",1040330


In [20]:
def genres(movie_id):
    genres=[]
  
    movie_details = tmdb_movie.details(movie_id)
    movie_genres = movie_details.genres
    if movie_genres:
          genre_str = " " 
          for i in range(0,len(movie_genres)):
              genres.append(movie_genres[i]['name'])
          return genre_str.join(genres)
    else:
          return np.NaN


In [21]:
df_2018['genres'] = df_2018['id'].map(lambda x: genres(int(x)))

<ipython-input-21-c1ea8b7e4402>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2018['genres'] = df_2018['id'].map(lambda x: genres(int(x)))


In [22]:
df_2018

,Title,Cast and crew,id,genres
0,Insidious: The Last Key,Adam Robitel (director); Leigh Whannell (scree...,406563,Horror Mystery Thriller
1,The Strange Ones,Lauren Wolkstein (director); Christopher Radcl...,426258,Thriller Drama
2,Stratton,"Simon West (director); Duncan Falconer, Warren...",348389,Action Thriller
3,Sweet Country,"Warwick Thornton (director); David Tranter, St...",468210,Drama Western Thriller
4,The Commuter,Jaume Collet-Serra (director); Byron Willinger...,399035,Action Thriller Mystery
...,...,...,...,...
269,Holmes & Watson,Etan Cohen (director/screenplay); Will Ferrell...,426563,Mystery Adventure Comedy Crime
270,Vice,Adam McKay (director/screenplay); Christian Ba...,82,Action Crime Thriller
271,On the Basis of Sex,Mimi Leder (director); Daniel Stiepleman (scre...,339380,Drama History
272,Destroyer,"Karyn Kusama (director); Phil Hay, Matt Manfre...",1040330,Documentary


In [23]:
def keywords(movie_id):
    keywords=[]

    movie_keywords = tmdb_movie.keywords(movie_id)
    #print(movie_keywords)
    if movie_keywords:
          #print(movie_keywords['keywords'])
          return movie_keywords['keywords']
    else:
          return np.NaN

In [24]:
df_2018['keywords'] = df_2018['id'].map(lambda x: keywords(int(x)))

<ipython-input-24-3d8575d9c279>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2018['keywords'] = df_2018['id'].map(lambda x: keywords(int(x)))


In [25]:
df_2018

,Title,Cast and crew,id,genres,keywords
0,Insidious: The Last Key,Adam Robitel (director); Leigh Whannell (scree...,406563,Horror Mystery Thriller,"[[id, name], [id, name], [id, name], [id, name..."
1,The Strange Ones,Lauren Wolkstein (director); Christopher Radcl...,426258,Thriller Drama,"[[id, name], [id, name], [id, name], [id, name..."
2,Stratton,"Simon West (director); Duncan Falconer, Warren...",348389,Action Thriller,"[[id, name], [id, name], [id, name]]"
3,Sweet Country,"Warwick Thornton (director); David Tranter, St...",468210,Drama Western Thriller,"[[id, name], [id, name], [id, name], [id, name..."
4,The Commuter,Jaume Collet-Serra (director); Byron Willinger...,399035,Action Thriller Mystery,"[[id, name], [id, name]]"
...,...,...,...,...,...
269,Holmes & Watson,Etan Cohen (director/screenplay); Will Ferrell...,426563,Mystery Adventure Comedy Crime,"[[id, name], [id, name], [id, name], [id, name..."
270,Vice,Adam McKay (director/screenplay); Christian Ba...,82,Action Crime Thriller,"[[id, name], [id, name], [id, name], [id, name..."
271,On the Basis of Sex,Mimi Leder (director); Daniel Stiepleman (scre...,339380,Drama History,"[[id, name], [id, name], [id, name], [id, name..."
272,Destroyer,"Karyn Kusama (director); Phil Hay, Matt Manfre...",1040330,Documentary,"[[id, name]]"


In [26]:
df_2018['keywords'] = df_2018['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

<ipython-input-26-13a170628011>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2018['keywords'] = df_2018['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])


In [27]:
df_2018

,Title,Cast and crew,id,genres,keywords
0,Insidious: The Last Key,Adam Robitel (director); Leigh Whannell (scree...,406563,Horror Mystery Thriller,"[medium, key, haunted house, prequel, spirit, ..."
1,The Strange Ones,Lauren Wolkstein (director); Christopher Radcl...,426258,Thriller Drama,"[sibling relationship, fire, cave, motel, nigh..."
2,Stratton,"Simon West (director); Duncan Falconer, Warren...",348389,Action Thriller,"[based on novel or book, terrorist, commando]"
3,Sweet Country,"Warwick Thornton (director); David Tranter, St...",468210,Drama Western Thriller,"[rape, slavery, court case, bible, australia, ..."
4,The Commuter,Jaume Collet-Serra (director); Byron Willinger...,399035,Action Thriller Mystery,"[conspiracy, train]"
...,...,...,...,...,...
269,Holmes & Watson,Etan Cohen (director/screenplay); Will Ferrell...,426563,Mystery Adventure Comedy Crime,"[england, judge, titanic, albert einstein, clu..."
270,Vice,Adam McKay (director/screenplay); Christian Ba...,82,Action Crime Thriller,"[miami, florida, florida, undercover, remake, ..."
271,On the Basis of Sex,Mimi Leder (director); Daniel Stiepleman (scre...,339380,Drama History,"[1970s, biography, supreme court, based on tru..."
272,Destroyer,"Karyn Kusama (director); Phil Hay, Matt Manfre...",1040330,Documentary,[behind the scenes]


In [28]:
s = df_2018.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'
s = s.value_counts()
s[:5]

<ipython-input-28-7aee9d8b2873>:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = df_2018.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)


based on novel or book    46
sequel                    30
based on true story       29
biography                 25
woman director            17
Name: keyword, dtype: int64

In [29]:
from ast import literal_eval
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet

In [30]:
s = s[s > 1]
stemmer = SnowballStemmer('english')

In [31]:
def keywordsFiltered(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [32]:
df_2018['keywords'] = df_2018['keywords'].apply(keywordsFiltered)
df_2018['keywords'] = df_2018['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])

<ipython-input-32-22221fb0bb17>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2018['keywords'] = df_2018['keywords'].apply(keywordsFiltered)
<ipython-input-32-22221fb0bb17>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2018['keywords'] = df_2018['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])


In [33]:
df_2018

,Title,Cast and crew,id,genres,keywords
0,Insidious: The Last Key,Adam Robitel (director); Leigh Whannell (scree...,406563,Horror Mystery Thriller,"[haunted hous, prequel, spirit, 1950s]"
1,The Strange Ones,Lauren Wolkstein (director); Christopher Radcl...,426258,Thriller Drama,"[sibling relationship, fire, motel, nightmar, ..."
2,Stratton,"Simon West (director); Duncan Falconer, Warren...",348389,Action Thriller,"[based on novel or book, terrorist]"
3,Sweet Country,"Warwick Thornton (director); David Tranter, St...",468210,Drama Western Thriller,"[rape, slaveri, court cas, bibl, based on true..."
4,The Commuter,Jaume Collet-Serra (director); Byron Willinger...,399035,Action Thriller Mystery,"[conspiraci, train]"
...,...,...,...,...,...
269,Holmes & Watson,Etan Cohen (director/screenplay); Will Ferrell...,426563,Mystery Adventure Comedy Crime,[sherlock holm]
270,Vice,Adam McKay (director/screenplay); Christian Ba...,82,Action Crime Thriller,"[florida, undercov, remak, drug deal, police d..."
271,On the Basis of Sex,Mimi Leder (director); Daniel Stiepleman (scre...,339380,Drama History,"[1970s, biographi, based on true stori, lawyer..."
272,Destroyer,"Karyn Kusama (director); Phil Hay, Matt Manfre...",1040330,Documentary,[]


In [34]:
df_2018['keywords'] = df_2018['keywords'].map(lambda x: " ".join(x))
df_2018['keywords']

<ipython-input-34-07107154fec3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2018['keywords'] = df_2018['keywords'].map(lambda x: " ".join(x))


0                      haunted hous prequel spirit 1950s
1      sibling relationship fire motel nightmar road ...
2                       based on novel or book terrorist
3      rape slaveri court cas bibl based on true stor...
4                                       conspiraci train
                             ...                        
269                                        sherlock holm
270    florida undercov remak drug deal police detect...
271    1970s biographi based on true stori lawyer wom...
272                                                     
273                                  video gam technolog
Name: keywords, Length: 274, dtype: object

In [35]:
def credits(movie_id):
    cast=[]
  
    movie_credits = tmdb_movie.credits(movie_id)
    if movie_credits['cast'] and movie_credits['crew']:
            for i in movie_credits['crew']:
                if i.get('job') == 'Director':
                    cast.append(i.get('name'))
                    break
            for i in range(0, len(movie_credits['cast'])):
                cast.append(movie_credits['cast'][i]['name'])
            return cast
    else:
            return np.NaN

In [36]:
df_2018['cast'] = df_2018['id'].map(lambda x: credits(int(x)))

<ipython-input-36-c9e5b0b20300>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2018['cast'] = df_2018['id'].map(lambda x: credits(int(x)))


In [37]:
df_2018

,Title,Cast and crew,id,genres,keywords,cast
0,Insidious: The Last Key,Adam Robitel (director); Leigh Whannell (scree...,406563,Horror Mystery Thriller,haunted hous prequel spirit 1950s,"[Adam Robitel, Lin Shaye, Angus Sampson, Leigh..."
1,The Strange Ones,Lauren Wolkstein (director); Christopher Radcl...,426258,Thriller Drama,sibling relationship fire motel nightmar road ...,"[Lauren Wolkstein, Alex Pettyfer, James Freeds..."
2,Stratton,"Simon West (director); Duncan Falconer, Warren...",348389,Action Thriller,based on novel or book terrorist,"[Simon West, Dominic Cooper, Gemma Chan, Conni..."
3,Sweet Country,"Warwick Thornton (director); David Tranter, St...",468210,Drama Western Thriller,rape slaveri court cas bibl based on true stor...,"[Warwick Thornton, Hamilton Morris, Bryan Brow..."
4,The Commuter,Jaume Collet-Serra (director); Byron Willinger...,399035,Action Thriller Mystery,conspiraci train,"[Jaume Collet-Serra, Liam Neeson, Vera Farmiga..."
...,...,...,...,...,...,...
269,Holmes & Watson,Etan Cohen (director/screenplay); Will Ferrell...,426563,Mystery Adventure Comedy Crime,sherlock holm,"[Etan Cohen, Will Ferrell, John C. Reilly, Reb..."
270,Vice,Adam McKay (director/screenplay); Christian Ba...,82,Action Crime Thriller,florida undercov remak drug deal police detect...,"[Michael Mann, Colin Farrell, Jamie Foxx, Gong..."
271,On the Basis of Sex,Mimi Leder (director); Daniel Stiepleman (scre...,339380,Drama History,1970s biographi based on true stori lawyer wom...,"[Mimi Leder, Felicity Jones, Armie Hammer, Jus..."
272,Destroyer,"Karyn Kusama (director); Phil Hay, Matt Manfre...",1040330,Documentary,,NaN


In [38]:
df_2018['cast'] = df_2018['cast'].apply(lambda x: [i for i in x] if isinstance(x, list) else [])
df_2018['cast'] = df_2018['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

<ipython-input-38-48654b00fe41>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2018['cast'] = df_2018['cast'].apply(lambda x: [i for i in x] if isinstance(x, list) else [])
<ipython-input-38-48654b00fe41>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2018['cast'] = df_2018['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])


In [39]:
df_2018

,Title,Cast and crew,id,genres,keywords,cast
0,Insidious: The Last Key,Adam Robitel (director); Leigh Whannell (scree...,406563,Horror Mystery Thriller,haunted hous prequel spirit 1950s,"[adamrobitel, linshaye, angussampson, leighwha..."
1,The Strange Ones,Lauren Wolkstein (director); Christopher Radcl...,426258,Thriller Drama,sibling relationship fire motel nightmar road ...,"[laurenwolkstein, alexpettyfer, jamesfreedson-..."
2,Stratton,"Simon West (director); Duncan Falconer, Warren...",348389,Action Thriller,based on novel or book terrorist,"[simonwest, dominiccooper, gemmachan, connieni..."
3,Sweet Country,"Warwick Thornton (director); David Tranter, St...",468210,Drama Western Thriller,rape slaveri court cas bibl based on true stor...,"[warwickthornton, hamiltonmorris, bryanbrown, ..."
4,The Commuter,Jaume Collet-Serra (director); Byron Willinger...,399035,Action Thriller Mystery,conspiraci train,"[jaumecollet-serra, liamneeson, verafarmiga, p..."
...,...,...,...,...,...,...
269,Holmes & Watson,Etan Cohen (director/screenplay); Will Ferrell...,426563,Mystery Adventure Comedy Crime,sherlock holm,"[etancohen, willferrell, johnc.reilly, rebecca..."
270,Vice,Adam McKay (director/screenplay); Christian Ba...,82,Action Crime Thriller,florida undercov remak drug deal police detect...,"[michaelmann, colinfarrell, jamiefoxx, gongli,..."
271,On the Basis of Sex,Mimi Leder (director); Daniel Stiepleman (scre...,339380,Drama History,1970s biographi based on true stori lawyer wom...,"[mimileder, felicityjones, armiehammer, justin..."
272,Destroyer,"Karyn Kusama (director); Phil Hay, Matt Manfre...",1040330,Documentary,,[]


In [40]:
df_2018['director'] = df_2018['cast'].map(lambda x:x[0] if len(x) >=1 else x)

In [41]:
df_2018

,Title,Cast and crew,id,genres,keywords,cast,director
0,Insidious: The Last Key,Adam Robitel (director); Leigh Whannell (scree...,406563,Horror Mystery Thriller,haunted hous prequel spirit 1950s,"[adamrobitel, linshaye, angussampson, leighwha...",adamrobitel
1,The Strange Ones,Lauren Wolkstein (director); Christopher Radcl...,426258,Thriller Drama,sibling relationship fire motel nightmar road ...,"[laurenwolkstein, alexpettyfer, jamesfreedson-...",laurenwolkstein
2,Stratton,"Simon West (director); Duncan Falconer, Warren...",348389,Action Thriller,based on novel or book terrorist,"[simonwest, dominiccooper, gemmachan, connieni...",simonwest
3,Sweet Country,"Warwick Thornton (director); David Tranter, St...",468210,Drama Western Thriller,rape slaveri court cas bibl based on true stor...,"[warwickthornton, hamiltonmorris, bryanbrown, ...",warwickthornton
4,The Commuter,Jaume Collet-Serra (director); Byron Willinger...,399035,Action Thriller Mystery,conspiraci train,"[jaumecollet-serra, liamneeson, verafarmiga, p...",jaumecollet-serra
...,...,...,...,...,...,...,...
269,Holmes & Watson,Etan Cohen (director/screenplay); Will Ferrell...,426563,Mystery Adventure Comedy Crime,sherlock holm,"[etancohen, willferrell, johnc.reilly, rebecca...",etancohen
270,Vice,Adam McKay (director/screenplay); Christian Ba...,82,Action Crime Thriller,florida undercov remak drug deal police detect...,"[michaelmann, colinfarrell, jamiefoxx, gongli,...",michaelmann
271,On the Basis of Sex,Mimi Leder (director); Daniel Stiepleman (scre...,339380,Drama History,1970s biographi based on true stori lawyer wom...,"[mimileder, felicityjones, armiehammer, justin...",mimileder
272,Destroyer,"Karyn Kusama (director); Phil Hay, Matt Manfre...",1040330,Documentary,,[],[]


In [42]:
df_2018['actor1']=df_2018['cast'].map(lambda x:x[1] if len(x) >=2 else x)
df_2018['actor2']=df_2018['cast'].map(lambda x:x[2] if len(x) >=3 else x)
df_2018['actor3']=df_2018['cast'].map(lambda x:x[3] if len(x) >=4 else x)

In [43]:
df_2018

,Title,Cast and crew,id,genres,keywords,cast,director,actor1,actor2,actor3
0,Insidious: The Last Key,Adam Robitel (director); Leigh Whannell (scree...,406563,Horror Mystery Thriller,haunted hous prequel spirit 1950s,"[adamrobitel, linshaye, angussampson, leighwha...",adamrobitel,linshaye,angussampson,leighwhannell
1,The Strange Ones,Lauren Wolkstein (director); Christopher Radcl...,426258,Thriller Drama,sibling relationship fire motel nightmar road ...,"[laurenwolkstein, alexpettyfer, jamesfreedson-...",laurenwolkstein,alexpettyfer,jamesfreedson-jackson,marinireland
2,Stratton,"Simon West (director); Duncan Falconer, Warren...",348389,Action Thriller,based on novel or book terrorist,"[simonwest, dominiccooper, gemmachan, connieni...",simonwest,dominiccooper,gemmachan,connienielsen
3,Sweet Country,"Warwick Thornton (director); David Tranter, St...",468210,Drama Western Thriller,rape slaveri court cas bibl based on true stor...,"[warwickthornton, hamiltonmorris, bryanbrown, ...",warwickthornton,hamiltonmorris,bryanbrown,samneill
4,The Commuter,Jaume Collet-Serra (director); Byron Willinger...,399035,Action Thriller Mystery,conspiraci train,"[jaumecollet-serra, liamneeson, verafarmiga, p...",jaumecollet-serra,liamneeson,verafarmiga,patrickwilson
...,...,...,...,...,...,...,...,...,...,...
269,Holmes & Watson,Etan Cohen (director/screenplay); Will Ferrell...,426563,Mystery Adventure Comedy Crime,sherlock holm,"[etancohen, willferrell, johnc.reilly, rebecca...",etancohen,willferrell,johnc.reilly,rebeccahall
270,Vice,Adam McKay (director/screenplay); Christian Ba...,82,Action Crime Thriller,florida undercov remak drug deal police detect...,"[michaelmann, colinfarrell, jamiefoxx, gongli,...",michaelmann,colinfarrell,jamiefoxx,gongli
271,On the Basis of Sex,Mimi Leder (director); Daniel Stiepleman (scre...,339380,Drama History,1970s biographi based on true stori lawyer wom...,"[mimileder, felicityjones, armiehammer, justin...",mimileder,felicityjones,armiehammer,justintheroux
272,Destroyer,"Karyn Kusama (director); Phil Hay, Matt Manfre...",1040330,Documentary,,[],[],[],[],[]


In [44]:
df_2018= df_2018.loc[:,['id','director','actor1','actor2','actor3','genres','keywords','Title']]

In [45]:
df_2018 = df_2018.rename(columns={'Title':'title'})

In [46]:
df_2018['title'] = df_2018['title'].str.lower()
df_2018['actor1'] = df_2018['actor1'].str.lower()
df_2018['actor2'] = df_2018['actor2'].str.lower()
df_2018['actor3'] = df_2018['actor3'].str.lower()
# data['director'] = data['director'].str.lower()
df_2018['comb'] = df_2018['genres'] + ' ' + df_2018['keywords'] + ' '  + df_2018['actor1'] + ' '  + df_2018['actor2'] + ' '  + df_2018['actor3'] + ' ' + df_2018['director']

In [47]:
df_2018

,id,director,actor1,actor2,actor3,genres,keywords,title,comb
0,406563,adamrobitel,linshaye,angussampson,leighwhannell,Horror Mystery Thriller,haunted hous prequel spirit 1950s,insidious: the last key,Horror Mystery Thriller haunted hous prequel s...
1,426258,laurenwolkstein,alexpettyfer,jamesfreedson-jackson,marinireland,Thriller Drama,sibling relationship fire motel nightmar road ...,the strange ones,Thriller Drama sibling relationship fire motel...
2,348389,simonwest,dominiccooper,gemmachan,connienielsen,Action Thriller,based on novel or book terrorist,stratton,Action Thriller based on novel or book terrori...
3,468210,warwickthornton,hamiltonmorris,bryanbrown,samneill,Drama Western Thriller,rape slaveri court cas bibl based on true stor...,sweet country,Drama Western Thriller rape slaveri court cas ...
4,399035,jaumecollet-serra,liamneeson,verafarmiga,patrickwilson,Action Thriller Mystery,conspiraci train,the commuter,Action Thriller Mystery conspiraci train liamn...
...,...,...,...,...,...,...,...,...,...
269,426563,etancohen,willferrell,johnc.reilly,rebeccahall,Mystery Adventure Comedy Crime,sherlock holm,holmes & watson,Mystery Adventure Comedy Crime sherlock holm w...
270,82,michaelmann,colinfarrell,jamiefoxx,gongli,Action Crime Thriller,florida undercov remak drug deal police detect...,vice,Action Crime Thriller florida undercov remak d...
271,339380,mimileder,felicityjones,armiehammer,justintheroux,Drama History,1970s biographi based on true stori lawyer wom...,on the basis of sex,Drama History 1970s biographi based on true st...
272,1040330,[],NaN,NaN,NaN,Documentary,,destroyer,NaN


In [48]:
df_2018.to_csv('movies2018.csv',index=False)

In [49]:
from google.colab import files
files.download("movies2018.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>